In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import display, Markdown, Latex
import pickle

SMALL_SIZE = 12
MEDIUM_SIZE = 14
LARGE_SIZE = 16

plt.rcdefaults()
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = MEDIUM_SIZE
plt.rcParams['axes.titlesize'] = LARGE_SIZE
plt.rcParams['axes.labelsize'] = MEDIUM_SIZE
plt.rcParams['xtick.labelsize'] = SMALL_SIZE
plt.rcParams['ytick.labelsize'] = SMALL_SIZE
plt.rcParams['legend.fontsize'] = MEDIUM_SIZE
plt.rcParams['figure.titlesize'] = LARGE_SIZE
plt.rcParams['figure.figsize'] = [7.2, 4.8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)

from utils import CobbDouglasDeflator

In [2]:
prob = CobbDouglasDeflator()

In [3]:
display(Latex(prob.general_setup()))
display(Latex(prob.general_solution()))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
display(Latex(prob.setup()))
print(prob.sol)
print("\nPeriod 1:")
print(prob.cb1.sol)
print("\nPeriod 2 with CPI adjustment:")
print(prob.cb2_cpi.sol)
print("\nPeriod 2 with Constant Utility adjustment:")
print(prob.cb2_cons.sol)

<IPython.core.display.Latex object>

{'CPI': 150.0, 'CONST_UTIL_DEFLATOR': 141.4213562373095}

Period 1:
{'x': 50.0, 'y': 50.0, 'U': 50.00000000000001, 'xmax': 100.0, 'ymax': 100.0}

Period 2 with CPI adjustment:
{'x': 75.0, 'y': 37.5, 'U': 53.033008588991066, 'xmax': 150.0, 'ymax': 75.0}

Period 2 with Constant Utility adjustment:
{'x': 70.71067811865476, 'y': 35.35533905932738, 'U': 50.0, 'xmax': 141.4213562373095, 'ymax': 70.71067811865476}


In [6]:
(1/8)**(-0.5)*50

141.4213562373095

In [4]:
examples = []
n_found = 0
for denom in [2, 3, 4]:
    for numer_a in range(1,denom):
        numer_b = denom - numer_a
        for px in range(1,11):
            for py in range(1,11):
                for I in np.array([10,12,100,120])*np.maximum(px,py):
                    params = {'numer_a':numer_a,'denom_a':denom,'numer_b':numer_b,'denom_b':denom,'I':I,'px':px,'py':py}
                    prob = CobbDouglasConsumer(params)
                    if prob.check_solution():
                        n_found+=1
                        examples.append( params | prob.sol )
                        if (n_found%1000==0):
                            print(f"{n_found} examples found...")
examples = pd.DataFrame.from_dict(examples)
examples.to_csv("CobbDouglasConsumer_examples.csv", index=False, header=True)

1000 examples found...


In [5]:
examples

,numer_a,denom_a,numer_b,denom_b,I,px,py,x,y,U,xmax,ymax
0,1,2,1,2,10,1,1,5.0,5.0,5.000000,10.0,10.0
1,1,2,1,2,12,1,1,6.0,6.0,6.000000,12.0,12.0
2,1,2,1,2,100,1,1,50.0,50.0,50.000000,100.0,100.0
3,1,2,1,2,120,1,1,60.0,60.0,60.000000,120.0,120.0
4,1,2,1,2,20,1,2,10.0,5.0,7.071068,20.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1227,3,4,1,4,120,10,6,9.0,5.0,7.770060,12.0,20.0
1228,3,4,1,4,1200,10,6,90.0,50.0,77.700602,120.0,200.0
1229,3,4,1,4,120,10,10,9.0,3.0,6.838521,12.0,12.0
1230,3,4,1,4,1000,10,10,75.0,25.0,56.987676,100.0,100.0


In [6]:
idx = np.random.randint(len(examples))
prob = CobbDouglasConsumer(dict(examples.loc[idx]))
display(Latex(prob.setup()))
print(prob.sol)

<IPython.core.display.Latex object>

{'x': 60.0, 'y': 60.0, 'U': 60.00000000000001, 'xmax': 120.0, 'ymax': 120.0}
